In [1]:
!pip install kaggle


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import kaggle

In [3]:
# download dataset using kaggle api

!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
# extract files from zipfiles
import zipfile
zip_ref=zipfile.ZipFile("orders.csv.zip")
zip_ref.extractall() # extract file to a directory
zip_ref.close()

In [5]:
# read data from csv and handle null values
import pandas as pd
df=pd.read_csv("orders.csv",na_values=['Not Available','unknown'])
df.head(20)

df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [6]:
# rename column names make lower case and replace space with underscore

df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(" ","_")
df.columns
df.head(5)

order_id  order_date       ship_mode    segment        country   
0         1  2023-03-01    Second Class   Consumer  United States  \
1         2  2023-08-15    Second Class   Consumer  United States   
2         3  2023-01-10    Second Class  Corporate  United States   
3         4  2022-06-18  Standard Class   Consumer  United States   
4         5  2022-07-13  Standard Class   Consumer  United States   

              city       state  postal_code region         category   
0        Henderson    Kentucky        42420  South        Furniture  \
1        Henderson    Kentucky        42420  South        Furniture   
2      Los Angeles  California        90036   West  Office Supplies   
3  Fort Lauderdale     Florida        33311  South        Furniture   
4  Fort Lauderdale     Florida        33311  South  Office Supplies   

  sub_category       product_id  cost_price  list_price  quantity   
0    Bookcases  FUR-BO-10001798         240         260         2  \
1       Chairs  FUR-CH-10000454         600         730         3   
2       Labels  OFF-LA-10000240          10          10         2   
3       Tables  FUR-TA-10000577         780         960         5   
4      Storage  OFF-ST-10000760          20          20         2   

   discount_percent  
0                 2  
1                 3  
2                 5  
3                 2  
4                 5

In [7]:
# derive new columns discount, sales price and profit

df['discount']=df['list_price']*df['discount_percent']*.01
df['sale_price']=df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']
df.head()

order_id  order_date       ship_mode    segment        country   
0         1  2023-03-01    Second Class   Consumer  United States  \
1         2  2023-08-15    Second Class   Consumer  United States   
2         3  2023-01-10    Second Class  Corporate  United States   
3         4  2022-06-18  Standard Class   Consumer  United States   
4         5  2022-07-13  Standard Class   Consumer  United States   

              city       state  postal_code region         category   
0        Henderson    Kentucky        42420  South        Furniture  \
1        Henderson    Kentucky        42420  South        Furniture   
2      Los Angeles  California        90036   West  Office Supplies   
3  Fort Lauderdale     Florida        33311  South        Furniture   
4  Fort Lauderdale     Florida        33311  South  Office Supplies   

  sub_category       product_id  cost_price  list_price  quantity   
0    Bookcases  FUR-BO-10001798         240         260         2  \
1       Chairs  FUR-CH-10000454         600         730         3   
2       Labels  OFF-LA-10000240          10          10         2   
3       Tables  FUR-TA-10000577         780         960         5   
4      Storage  OFF-ST-10000760          20          20         2   

   discount_percent  discount  sale_price  profit  
0                 2       5.2       254.8    14.8  
1                 3      21.9       708.1   108.1  
2                 5       0.5         9.5    -0.5  
3                 2      19.2       940.8   160.8  
4                 5       1.0        19.0    -1.0

In [8]:
# convert order date from object data type to datetime
# df.dtypes
df["order_date"]=pd.to_datetime(df['order_date'],format="%Y-%m-%d")
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [9]:
# drop cost price list price and discount percent columns

df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)

In [10]:
!pip install sqlalchemy


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install sqlalchemy==1.4.31

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# load data into sql server using replace option
import sqlalchemy as sal
engine=sal.create_engine("mssql://JYOTI\SQLEXPRESS/dataDB?driver=ODBC+DRIVER+17+FOR+SQL+SERVER")
conn=engine.connect()

In [21]:
# load the data to sql server using replace option
df.to_sql('df_orders',con=conn, index=False,if_exists='append')

-1